#### include library code

In [ ]:
using Random

#### define agents

In [ ]:
mutable struct ComplexHuman
    # var :: type
    migrant :: Bool
    employed :: Bool
    industry :: Int
    origin :: Country
    residence :: Country
    contacts :: Vector{ComplexHuman}
end

#### define countries

In [ ]:
mutable struct Country
    # this rate w/ be used in migration decision
    migration_rate :: Float64 # or should it be in the simulation structure?
    job_market :: Int
    industry :: Vector{Industry}
end

#### define industries and job market

In [ ]:
mutable struct Industry
    num_jobs :: Int
     # job hire rate
    hirer :: Float64
    # being fired
    firer :: Float64
    # communication rate
#     commr :: Float64
end
function Industry() 
    Industry(0,0,0)
end

#### define simulation
---
but I'm not sure it's what we would like to have
***
Actually I'm quite sure not, but I don't know how to turn *this* into the simulation we'd like to have


In [3]:
mutable struct Simulation
    # model parameters:
    industry :: Vector{Industry}
    # communication rate
    commr :: Float64
    # and this is our population of agents
    pop :: Vector{ComplexHuman}
end

#### updating functions

In [ ]:
function update_migrant_status!(person, sim)
     if person.migrant == true
        return
    end
    if person.migrant == false
        if person.contacts.migrant == true && person.contacts.employed >= "friends_employment_rate" && rand() < sim.commr
#             ...I'd like to check how many of contacts are employed. if greater then a certain amount THEN
            person.migrant == true
            person.residence == rand(person.contacts).residence
        end
    end
end



function update_migrant_employment!(person, sim)
    if person.employed == true
        return
    end
    if person.employed == false
        if person.contacts.migrant == true && person.contacts.employed == true && person.contacts.industry # are employed in a certain jobmarket with a rate higher than a certain amount THEN
            if rand()< industry.hire.rate # same industry of his friends
            person.employed == true
            person.industry == rand(person.contacts).industry
            end
        end
    end
end

function update_pop!(pop, sim)  
    for i in pop
        update_person!(i)
    end
end


function update_person(agent, sim)
    update_migrant_status!(agent, sim)
    update_migrant_employment!(agent, sim)
end

function update_country!()
    update_industry!()
end


## firing function


# function update_job_market!(industry, pop)  # needs to be like hey no more jobs --- you can't get these jobs they're gone
#     for i in length(pop):
#         if pop[i].employed == false
#             return
#         end
#         if pop[i].employed == true
#             industry.num_jobs += 1
#         end
#     end
# end

#### setup functions

In [ ]:
function setup_industries(n, industry_jobs,glob_hirer, glob_firer, country)
    country.industry = [Industry(rand()*industry_jobs,glob_hirer+rand()*0.2-rand()*0.2,glob_firer+rand()*0.2-rand()*0.2 ) for i=1:n ]
end

function setup_countries(n, ) # change values alike!
    sim.country = [Industry(rand()*industry_jobs,glob_hirer+rand()*0.2-rand()*0.2,glob_firer+rand()*0.2-rand()*0.2 ) for i=1:n]
    for country in sim.countries
        setup_industries(n, industry_jobs,glob_hirer, glob_firer, country)
    end
end


    # go through all combinations of agents and 
    # check if they are connected
    for i in eachindex(pop)
        for j in i+1:length(pop)
            if rand() < p_contact
                push!(pop[i].contacts, pop[j])
                push!(pop[j].contacts, pop[i])
            end
        end
    end

    pop
end

#### run

In [ ]:
COLLECT DATA
RUN UPDATE FUNCTS


function run_sim(sim, n_steps, verbose = false)
    # we keep track of the numbers
    n_inf = Int[]
    n_susc = Int[]

    # simulation steps
    for t in  1:n_steps
        update_agents!(sim)
        push!(n_inf, count(p -> p.status == infected, sim.pop))
        push!(n_susc, count(p -> p.status == susceptible, sim.pop))
        # a bit of output
        if verbose
            println(t, ", ", n_inf[end], ", ", n_susc[end])
        end
    end
end


FOR PLOT LOOK AT PLOT 0

IN RUN FUNCTION CALCULATE AN ARRAY OF NUMBER OF INTERESTS AN PLOT THAT ONE.